<span type="title">SSH Integration</span> | <span type="update">2018-10-26</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Spring、Spring MVC、Hibernate 的整合。</p><p class="card-text">在整合  Spring 和 Hibernate 的时候，尤其需要注意：① 数据源：数据源涉及认证信息，应当采用 properties 动态加载。应当至少提供数据源的用户名、密码、地址（数据库名）以及基本增长策略和限制。② Spring 托管的 Session 对象：尤其注意，不能使用 thread 方式，应当使用 Spring 提供的 SpringSessionContext，使用 getCurrentSession API 获取线程绑定的 Session。③ 持久化层（DAO和持久化类）应当和 Spring 完全解耦，服务层应当和 Hibernate 完全解耦，即所有服务都要经过 Service 操纵 DAO 的 API 来进行。</p><p class="card-text">在整合 Spring 和 Spring MVC 的时候，需要进行 Spring IOC 的监听和自动绑定，还需要注意： ① Spring 自动注解扫描一定要排除 MVC 子 IOC 容器的路径，否则容易造成配置复写和资源浪费（一般情况下过滤掉Controller和 ControllerAdvice注解即可）。② 配置文件路径需要注意 classpath 和物理路径的问题，尤其是之前在 Spring 中配置的 Hibernate 映射文件路径问题，看清楚是类路径还是根目录。</p></span>

在经典的 Hibernate 程序中，采用 hibernate.cfg.xml 配置文件配置数据库、数据源、域模型相关策略（显示 SQL 语句、格式化 SQL 语句，开启二级缓存、查询缓存、启用数据库方言等等）。整合 Hibernate 的目的在于，让 WEB 程序能够快速的通过 Hibernate 的 SessionFactory 获得 Session，并且进行数据交互操作。在 Hibernate 的章节，我们介绍了基于 thread 的线程绑定获取 Session 的方式。这常被直接写入 Data Access Object 中，作为类实例变量，进行数据交互使用。

因此，我们面临的唯一问题就是在何时获得 SessionFactory 对象，一般而言，我们使用 Spring IOC 容器来管理 Hibernate 的 SessionFactory。很显然的，IOC 中创建 Bean 的方式创建 SF，可选两种方案，其一，将所有原本 Hibernate 配置信息作为参数传递进去，其二，将原本 Hibernate 配置文件保留，让 SessionFactory 对象自动寻找 Hibernate 配置文件，并且进行加载。

SSH 整体概要如下：

```
                                |--------------------- Spring IOC ---------------------------|
                                |                      |-- Spring AOP --|-- Spring MVC IOC --|
   |-- database.properties      |                      |                |                    |
Database ------ Entities ------ Data Access Object ------ Service ------ Controller ------ Servlet ------ View
   |             |--entities.hbm.xml        |                |               |               |-- web.xml    |
  JDBC           |------hibernate.cfg.xml   |             Exception          |-- spring-mvc.xml             |     
   |----------------Hibernate---------------|                                |-----------Spring MVC --------|  
```

# 在 IOC 中配置持久化层

一个整合 Spring 的 Hiberante 项目文件配置如下：

```
resources
|--- hbm
|     |---entities.hbm.xml
|--- spring.cfg.xml
|--- hibernate.cfg.xml
|--- spring-mvc.cfg.xml
|--- database.properties
```

## DataSource 和 SessionFactory 的配置

其 Hibernate.cfg.xml 如下：

```xml
<?xml version='1.0' encoding='utf-8'?>
<!DOCTYPE hibernate-configuration PUBLIC "-//Hibernate/Hibernate Configuration DTD//EN"
        "http://www.hibernate.org/dtd/hibernate-configuration-3.0.dtd">
<hibernate-configuration>
    <session-factory>
        <property name="hibernate.dialect">org.hibernate.dialect.MySQL8Dialect</property>
        <property name="hibernate.show_sql">true</property>
        <property name="hibernate.hbm2ddl.auto">update</property>
        <property name="hibernate.current_session_context_class">
        org.springframework.orm.hibernate5.SpringSessionContext</property>
    </session-factory>
</hibernate-configuration>
```

其中 DataSource 数据源可能除了 Hibernate，还有别的用途，因此需要配置到 IOC 容器中，然后作为参数传递给 Session Factory 的 bean 对象。因此，整合后的 Hibernate 配置文件仅仅配置：方言、SQL 显示、格式化、生成数据表的策略和二级缓存策略。

一个整合了 Hibernate 的 Spring.cfg.xml 文件如下所示，注意，这里需要配置数据源、SessionFactory 以及配置交给 Spring 管理的 Hibernate 的事务。

对于数据源而言，一般提供一个 properties 文件，放置数据敏感信息，然后使用占位符来加载信息，之后向 beans 中写入信息。数据源需要注意的是，<u>忘记写数据库地址会报找不到数据库</u>错误。

对于 SessionFacotry 而言，我们需要提供数据源 bean 引用，配置文件路径，映射模型文件路径，除了使用配置文件以外，还可以直接在 hibernateProperties 属性中提供 props 的标签，写入各个原本 Hibernate 配置的字段，作为 key - value 传入，如果同时加载了配置，则以此处为准。<u>配置此处需要注意，路径需要区分 classpath 和物理目录。对于 maven 工程，一般放置在 resource 目录下，也就是作为 classpath 路径下的目录。当进行 web 整合的时候，如果放置在此文件夹下的目录不添加 classpath 前缀，则容易发生找不到文件的异常。</u>

对于数据库方言，需要注意，如果默认使用了 MySQL5Dialect，则使用的引擎 MyISDM 不支持事务。

其 spring.cfg.xml 如下：

```xml
<?xml version="1.0" encoding="UTF-8"?>
<beans xmlns="http://www.springframework.org/schema/beans"
       xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:beans="http://www.springframework.org/schema/beans"
       xmlns:context="http://www.springframework.org/schema/context" xmlns:p="http://www.springframework.org/schema/p"
       xmlns:tx="http://www.springframework.org/schema/tx" xmlns:aop="http://www.springframework.org/schema/aop"
       xsi:schemaLocation="http://www.springframework.org/schema/beans
       http://www.springframework.org/schema/beans/spring-beans.xsd
       http://www.springframework.org/schema/context http://www.springframework.org/schema/context/spring-context.xsd
       http://www.springframework.org/schema/tx http://www.springframework.org/schema/tx/spring-tx.xsd
       http://www.springframework.org/schema/aop http://www.springframework.org/schema/aop/spring-aop.xsd
">
    <!--在此处导入 DataSource 而不是 Hibernate 中, 使用 Java Class 而不是 C3P0.xml 配置-->
    <context:property-placeholder location="classpath:database.properties" />
    <bean class="com.mchange.v2.c3p0.ComboPooledDataSource" id="dataSource"
          p:user="${jdbc.user}" p:password="${jdbc.password}" p:driverClass="${jdbc.driverClass}"
          p:initialPoolSize="${jdbc.initPoolSize}" p:maxPoolSize="${jdbc.maxPoolSize}"
          p:jdbcUrl="${jdbc.jdbcUrl}"
    />
    <!--在此处配置 Hibernate 的 SessionFactory， 同时提供其配置文件和映射文件-->
    <bean class="org.springframework.orm.hibernate5.LocalSessionFactoryBean" id="sessionFactory"
          p:dataSource-ref="dataSource"
          p:configLocation="classpath:hibernate.cfg.xml"
          p:mappingLocations="classpath:hbm/*.hbm.xml">
        <property name="hibernateProperties">
            <props>
                <prop key="hibernate.dialect">org.hibernate.dialect.MySQL8Dialect</prop>
                <prop key="hibernate.show_sql">true</prop>
                <prop key="hibernate.format_sql">false</prop>
                <prop key="hibernate.hbm2ddl.auto">update</prop>
            </props>
        </property>
    </bean>
    <!--配置 Spring 声明式事务：事务管理器、事务属性、事务切点-->
    <bean class="org.springframework.orm.hibernate5.HibernateTransactionManager" id="transactionManager"
          p:sessionFactory-ref="sessionFactory"/>
    <tx:advice id="txAdvice">
        <tx:attributes>
            <tx:method name="get*" read-only="true"/>
            <tx:method name="purchase" propagation="REQUIRES_NEW"/>
            <tx:method name="*" />
        </tx:attributes>
    </tx:advice>
    <aop:config>
        <aop:pointcut id="txPointcut" expression="execution(* com.mazhangjing.learn.temp.service.*.*(..))"/>
        <aop:advisor advice-ref="txAdvice" pointcut-ref="txPointcut" />
    </aop:config>

    <context:component-scan base-package="com.mazhangjing.learn.temp">
        <context:exclude-filter type="annotation" expression="org.springframework.stereotype.Controller" />
        <context:exclude-filter type="annotation" expression="org.springframework.web.bind.annotation.ControllerAdvice" />
    </context:component-scan>
</beans>
```

尤其需要注意，结合了 Spring 的 Hibernate 配置 key 的前缀必须以 hibernate 开头，而不能只写原本在 hibernate 中的 key 名称。

## Spring 声明式事务的配置

除了配置数据源、提供数据源、映射文件、配置文件以配置 SessionFactory，还需要配置声明式事务支持。因为主要采用了 XML 方式进行配置，因此，这里直接使用手动的方式，而不是基于注解，来配置事务。

事务需要提供事务管理器，此管理器实现了对于事务管理在 Spring 的实现，此外，需要提供 tx:advice 标签来定义此管理器的策略组。在此组中，需要配置 tx:method 来指定方法的事务的策略，常见的 get*, 一般选择只读来优化查询性能。对于特殊的服务层方法，则直接定义 tx:method 方法，比如指定事务合并级别：开新事务，而不是默认的合并入他人的事务中。此外，使用星号对于其余的事务进行默认的配置。

当我们有了事务管理器，并且配置好事务策略后，需要通过 aop:config 标签定义事务切入的位点：一般需要作用于 service 层，使用 `* xxx.service.*.*(..)` 来作用于 service 包下所有类的所有方法。使用 aop:pointcut 来定义切点，使用 aop:advisor 来应用事务策略组。

## Spring 启用自动注解扫描

一般而言，我们直接对所有的包以及子包启用注解扫描，以将其添加到 IOC 中。但是，整合的 Spring 框架需要小心避免 MVC 和 Spring IOC 框架冲突问题，对于 Spring IOC 而言，其是作为父容器存在的，因此自动扫描注解较为宽松，开启默认过滤器，允许全部包，然后排除 Cotroller 和 ControllerAdvice 注解即可，因为这两个注解分别用于控制层的 URL 映射以及映射错误处理，需要让 MVC 在其子 IOC 容器中实现。


## Hibernate 类模型文件的配置

其 entities.hbm.xml 映射文件配置如下：

```xml
<?xml version='1.0' encoding='utf-8'?>
<!DOCTYPE hibernate-mapping PUBLIC
        "-//Hibernate/Hibernate Mapping DTD 3.0//EN"
        "http://www.hibernate.org/dtd/hibernate-mapping-3.0.dtd">
<hibernate-mapping package="com.mazhangjing.learn.temp.entities">
    <class name="Account" table="ACCOUNT">
        <id name="id" type="integer" column="ID">
            <generator class="native"/>
        </id>
        <property name="username" type="string" column="USERNAME"/>
        <property name="balance" type="int" column="BALANCE" />
    </class>
    <class name="Book" table="BOOK">
        <id name="id" type="integer" column="BOOK_ID">
            <generator class="native"/>
        </id>
        <property name="bookName" type="string" column="BOOK_NAME" />
        <property name="isbn" type="string" column="ISBN" />
        <property name="price" type="integer" column="PRICE" />
        <property name="stock" type="integer" column="STOCK" />
    </class>
</hibernate-mapping>
```

以上是所有的配置文件， 包括持久化层的映射模型、持久化层的全局配置、Spring IOC 的整合配置文件（供服务层调用），此外，还有 Spring MVC 的控制层配置文件。

# 基于 Hibernate 的服务层的实现

```
java
|--- entities
|     |--- Account.java
|     |--- Book.java
|--- dao
|     |--- BookShopDao.java
|--- service
|     |--- BookShopService.java
|     |--- Cashier.java
|--- exception
|     |--- AccountMoneyException.java
|     |--- BookStockException.java
|--- controller
|     |--- HomeController.java
```
其中持久化层为 entities 和 dao，服务层为 service，其错误信息为 exception，控制层为 controller。

## 持久化 POJO 类的实现

```java
/com.mazhangjing.spring.entities
public class Account {
    private Integer id;
    private String username;
    private int balance;
    ...
}
public class Book {
    private Integer id;
    private String bookName;
    private String isbn;
    private int price;
    private int stock;
    ...
}
```

## 持久化层 DAO 的实现

```java
/com.mazhangjing.spring.dao
@Repository
public class BookShopDao {

    @Autowired private SessionFactory sessionFactory;

    private Session getSession() {
        return sessionFactory.getCurrentSession();
    }

    public BookShopDao() {
        System.out.println(getClass().getSimpleName() + "is init.....");
    }

    public int findBookPriceByIsbn(String isbn) {
        String hql = "select b.price from Book b where b.isbn = ?1";
        Query query = getSession().createQuery(hql).setParameter(1,isbn);
        return (int) query.uniqueResult();
    }

    public void updateBookStock(String isbn) {
        String hql_s = "select b.stock from Book b where b.isbn = ?1";
        int stock = (int) getSession().createQuery(hql_s).setParameter(1,isbn).uniqueResult();
        if (stock == 0) throw new BookStockException("库存不足");
        String hql = "update Book b set b.stock = b.stock - 1 where b.isbn = ?1";
        getSession().createQuery(hql).setParameter(1,isbn).executeUpdate();
    }
    public void updateUserAccount(String username, int price) {
        String check = "select a.balance from Account a where a.username = ?1";
        int balance = (int) getSession().createQuery(check).setParameter(1,username).uniqueResult();
        if (balance < price) throw new AccountMoneyException("余额不足");

        String hql = "update Account a set a.balance = a.balance - ?1 where a.username = ?2";
        Query query = getSession().createQuery(hql).setParameter(1, price).setParameter(2, username);
        query.executeUpdate();
    }
}
```

对于持久化层，需要添加为 bean，此层保留 sessionFactory 的类实例，通过自动注入获得。其提供和数据进行交互的各种方法，在内部采用 Hibernate 提供的 API 来进行数据操作，这里不需要捕获异常。

需要注意，此层和 Hibernate 耦合，但是不要和 Service、Spring 进行耦合。这样方便数据存储层的复用和移植。注意，这里使用了 `sessionFactory.getCurrentSession();` 来获取 Session，其等同于 基于 thread 的 Session 获取方式，不过不同的是，基于 thread 的 Session 获取是为 JDBC 服务的，我们需要将 Session 交给 Spring，让其负责进行事务管理。因此，这里默认不写 Session 获取方式的配置，如果实在要配，其实是这个：

```
<property name="hibernate.current_session_context_class">
        org.springframework.orm.hibernate5.SpringSessionContext</property>
```

这里通过本地线程从 IOC 容器初始化好的 SessionFactory 的 getCurrentSession 拿到 Session 对象，进行持久化层的操作。


## 服务层的实现

```java
/com.mazhangjing.spring.service
@Service
public class BookShopService {

    private final BookShopDao bookShopDao;

    @Autowired public BookShopService(BookShopDao bookShopDao) { this.bookShopDao = bookShopDao;
        System.out.println(getClass().getSimpleName() + "is init.....");}

    public void purchase(String username, String isbn) {
        int price = bookShopDao.findBookPriceByIsbn(isbn);
        bookShopDao.updateBookStock(isbn);
        bookShopDao.updateUserAccount(username, price);
    }
}
```

这里的服务层调用持久层的 DAO API 进行数据操纵，在这里进行了 DAO 的 bean 注入。注意，服务层不涉及任何涉及持久层的代码和配置，包括任何 Hibernate 的代码。此处将会被自动切入，并且提供事务支持（在配置文件中设置了为 service 包添加事务切面）。

注意，这里也可以不使用 tx:advice 配置事务，而是采用 @Transactional 注解提供事务支持，如果采用后者，需要提供 `<tx:annotation-driven />` 标签。

# IOC 容器和控制层的整合

我们需要在 WEB 服务启动的时候自动装入 Spring IOC，采用监听器即可完成。Spring 默认提供了一个监听器，只用传入 Spring 配置文件，即可自动在 WEB 服务启动时加载 Spring IOC 容器。

```xml
/WEB-INF/web.xml
<?xml version="1.0" encoding="UTF-8"?>
<web-app xmlns="http://xmlns.jcp.org/xml/ns/javaee"
         xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
         xsi:schemaLocation="http://xmlns.jcp.org/xml/ns/javaee
                      http://xmlns.jcp.org/xml/ns/javaee/web-app_4_0.xsd"
         version="4.0">

    <context-param>
        <param-name>contextConfigLocation</param-name>
        <param-value>classpath:spring.cfg.xml</param-value>
    </context-param>
    <listener>
        <listener-class>org.springframework.web.context.ContextLoaderListener</listener-class>
    </listener>

    <filter>
        <filter-name>methodFilter</filter-name>
        <filter-class>org.springframework.web.filter.HiddenHttpMethodFilter</filter-class>
    </filter>
    <filter-mapping>
        <filter-name>methodFilter</filter-name>
        <url-pattern>/*</url-pattern>
    </filter-mapping>

    <servlet>
        <servlet-name>mvc</servlet-name>
        <servlet-class>org.springframework.web.servlet.DispatcherServlet</servlet-class>
        <init-param>
            <param-name>contextConfigLocation</param-name>
            <param-value>classpath*:spring-mvc.cfg.xml</param-value>
        </init-param>
    </servlet>
    <servlet-mapping>
        <servlet-name>mvc</servlet-name>
        <url-pattern>/</url-pattern>
    </servlet-mapping>
    <jsp-config>
        <jsp-property-group>
            <url-pattern>*.jsp</url-pattern>
            <trim-directive-whitespaces>true</trim-directive-whitespaces>
        </jsp-property-group>
    </jsp-config>
</web-app>
```

以上是基本的 web.xml 配置，包括了 RESTFUL 风格的 CRUD 的 HTTP 方法过滤器，自动加载 Spring IOC 的监听器，注意这里我们提供了 context-param，传递了 Spring 的配置文件，这里尤其需要注意 classpath 的问题。之后就是传统的 URL 截获，交给 DispatcherServlet 进行所有控制层的 URL 处理。

Ps. 这里需要注意，启用 jsp 配置文件过滤空白符后，jsp 配置组的 `/*` URL 路径将会影响 Servlet 的映射，应当为 `*.jsp`。

以下是 控制层的配置文件：

为了简化起见，这里的控制层仅仅配置了注解自动扫描，映射自动扫描，静态文件过滤，添加了基本的 JSP 视图层处理器。这里需要注意，Spring MVC IOC 容器是 Spring IOC 容器的子容器，因此，需要关闭默认过滤器，只允许 Controller 和 ControllerAdvice 注解加载为子容器的 beans。

```xml
<?xml version="1.0" encoding="UTF-8"?>
<beans xmlns="http://www.springframework.org/schema/beans"
       xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
       xmlns:beans="http://www.springframework.org/schema/beans"
       xmlns:context="http://www.springframework.org/schema/context"
       xmlns:mvc="http://www.springframework.org/schema/mvc" xmlns:p="http://www.springframework.org/schema/p"
       xsi:schemaLocation="http://www.springframework.org/schema/beans
       http://www.springframework.org/schema/beans/spring-beans.xsd
       http://www.springframework.org/schema/context
       http://www.springframework.org/schema/context/spring-context.xsd
       http://www.springframework.org/schema/mvc
       http://www.springframework.org/schema/mvc/spring-mvc.xsd
">
    <context:component-scan base-package="com.mazhangjing.learn.temp.*" use-default-filters="false">
        <context:include-filter type="annotation" expression="org.springframework.stereotype.Controller"/>
        <context:include-filter type="annotation" expression="org.springframework.web.bind.annotation.ControllerAdvice" />
    </context:component-scan>
    <mvc:default-servlet-handler />
    <mvc:annotation-driven />
    <bean class="org.springframework.web.servlet.view.InternalResourceViewResolver" id="resourceViewResolver"
          p:suffix=".jsp" p:prefix="/WEB-INF/views/"/>
</beans>
```

以下是控制层的映射的实现：

```java
/com.mazhangjing.spring.controller
@Controller
@RequestMapping("/")
public class HomeController {
    @Autowired ApplicationContext context;
    @RequestMapping("/")
    public String home(){
        BookShopService bookShopService = context.getBean(BookShopService.class);
        bookShopService.purchase("TEST","1001");
        return "index";
    }
}
```

可以看到，我们自动加入的 Spring IOC 容器，可以通过 @Autowire 被 Spring MVC IOC 容器进行自动装配，这非常方便。此外，如果不希望自动装配，通过 WebApplicationContextUtils 的 getWebApplicaitonContext 获取此 context，然后进一步获取对象即可。

```jsp
/WEB-INF/views/index.jsp
<% ApplicationContext context = WebApplicationContextUtils.getWebApplicationContext(application);
DataSource bean = context.getBean(DataSource.class);
request.setAttribute("bean",bean); %>

<p>Your data source is: ${bean}</p>
```

# SSH 的 MAVEN 依赖

SSH 的依赖问题可以分为几个方面：

## Java WEB 的依赖

包括 Servlet、JSP、JSTL、EL 等，需要注意，因为此处的包，如果和 TOMCAT 的包不一致，则极其容易造成冲突，因此，此处的作用域只应当为 provied。

```xml
<!--JavaEE Web Support-->
<!-- https://mvnrepository.com/artifact/javax.servlet/javax.servlet-api -->
<dependency>
    <groupId>javax.servlet</groupId>
    <artifactId>javax.servlet-api</artifactId>
    <version>4.0.1</version>
    <scope>provided</scope>
</dependency>
<!-- https://mvnrepository.com/artifact/javax.servlet.jsp/javax.servlet.jsp-api -->
<dependency>
    <groupId>javax.servlet.jsp</groupId>
    <artifactId>javax.servlet.jsp-api</artifactId>
    <version>2.3.3</version>
    <scope>provided</scope>
</dependency>
<!-- https://mvnrepository.com/artifact/javax.servlet.jsp.jstl/jstl -->
<dependency>
    <groupId>javax.servlet.jsp.jstl</groupId>
    <artifactId>jstl</artifactId>
    <version>1.2</version>
    <scope>provided</scope>
</dependency>
<!-- https://mvnrepository.com/artifact/javax.el/javax.el-api -->
<dependency>
    <groupId>javax.el</groupId>
    <artifactId>javax.el-api</artifactId>
    <version>3.0.0</version>
    <scope>provided</scope>
</dependency>
```

## Spring IOC/AOP 的依赖

最大的问题就是 AspectJ 两件套了(aspectjweaver, aspectjrt)

```xml
<!--Spring IOC/AOP-->
<!-- https://mvnrepository.com/artifact/org.springframework/spring-context -->
<dependency>
    <groupId>org.springframework</groupId>
    <artifactId>spring-context</artifactId>
    <version>${spring_version}</version>
</dependency>
<!-- https://mvnrepository.com/artifact/org.springframework/spring-core -->
<dependency>
    <groupId>org.springframework</groupId>
    <artifactId>spring-core</artifactId>
    <version>${spring_version}</version>
</dependency>
<!-- https://mvnrepository.com/artifact/org.aspectj/aspectjrt -->
<dependency>
    <groupId>org.aspectj</groupId>
    <artifactId>aspectjrt</artifactId>
    <version>${aspectj_version}</version>
</dependency>
<!-- https://mvnrepository.com/artifact/org.aspectj/aspectjweaver -->
<dependency>
    <groupId>org.aspectj</groupId>
    <artifactId>aspectjweaver</artifactId>
    <version>${aspectj_version}</version>
</dependency>
```

## SpringMVC 的依赖

如果控制层还需要使用 Hibernate Valiator 验证等，还需要添加相应的 jar 包。

```xml
<!--Spring MVC-->
<!-- https://mvnrepository.com/artifact/org.springframework/spring-web -->
<dependency>
    <groupId>org.springframework</groupId>
    <artifactId>spring-web</artifactId>
    <version>${spring_version}</version>
</dependency>
<!-- https://mvnrepository.com/artifact/org.springframework/spring-webmvc -->
<dependency>
    <groupId>org.springframework</groupId>
    <artifactId>spring-webmvc</artifactId>
    <version>${spring_version}</version>
</dependency>
```

## Hibernate 的依赖

Hibernate 需要提供 其自身、数据库、数据源的配置

```xml
<!--Hibernate in Spring with C3PO and MySQL-->
<!-- https://mvnrepository.com/artifact/org.springframework/spring-orm -->
<dependency>
    <groupId>org.springframework</groupId>
    <artifactId>spring-orm</artifactId>
    <version>${spring_version}</version>
</dependency>
<!-- https://mvnrepository.com/artifact/org.springframework/spring-tx -->
<dependency>
    <groupId>org.springframework</groupId>
    <artifactId>spring-tx</artifactId>
    <version>${spring_version}</version>
</dependency>
<!-- https://mvnrepository.com/artifact/mysql/mysql-connector-java -->
<dependency>
    <groupId>mysql</groupId>
    <artifactId>mysql-connector-java</artifactId>
    <version>${mysql_version}</version>
</dependency>
<!-- https://mvnrepository.com/artifact/com.mchange/c3p0 -->
<dependency>
    <groupId>com.mchange</groupId>
    <artifactId>c3p0</artifactId>
    <version>0.9.5.2</version>
</dependency>
<!-- https://mvnrepository.com/artifact/org.hibernate/hibernate-core -->
<dependency>
    <groupId>org.hibernate</groupId>
    <artifactId>hibernate-core</artifactId>
    <version>${hibernate_version}</version>
</dependency>
```

最后是 JUnit：

```xml
<!-- https://mvnrepository.com/artifact/junit/junit -->
<dependency>
    <groupId>junit</groupId>
    <artifactId>junit</artifactId>
    <version>4.12</version>
    <scope>test</scope>
</dependency>
```

目前最新的配置为：

```xml
<properties>
    <spring_version>5.1.1.RELEASE</spring_version>
    <aspectj_version>1.9.1</aspectj_version>
    <mysql_version>8.0.13</mysql_version>
    <hibernate_version>5.3.7.Final</hibernate_version>
</properties>
```